In [35]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pyreadstat as st
import numpy as np
import matplotlib.pyplot as plt


path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 0-20250420\ps0_data\sim_1.dta"

df, meta = st.read_dta(path)
df

,x2,x3,e,u,y
0,6.954513,4.884080,-5.819671,-7.075903,5.762690
1,8.537341,-4.902815,3.898010,5.552234,10.186760
2,1.739996,9.951114,-5.602970,-4.044251,8.646858
3,-3.906954,-3.913731,5.773367,2.369219,-4.451466
4,2.368236,-2.692694,4.975786,3.824420,4.499962
5,-3.726072,-0.859068,-7.732616,-3.231157,-6.816297
6,3.618390,3.942587,3.645388,3.174982,11.735958
7,9.037177,2.183038,-2.760063,-4.132867,8.087349
8,4.197956,1.401212,2.532652,2.337455,8.936624
9,2.557375,-2.210001,-2.918557,-2.286053,-0.938679


In [4]:
import statsmodels.api as sm

y = df['y']
x = df[['x2', 'x3']]
X = sm.add_constant(x)

model = sm.OLS(y,X)
results = model.fit()

results.summary()

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     8.698
Date:                Wed, 07 May 2025   Prob (F-statistic):             0.0127
Time:                        21:09:30   Log-Likelihood:                -26.052
No. Observations:                  10   AIC:                             58.10
Df Residuals:                       7   BIC:                             59.01
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9142      1.542      0.593      0.572      -2.731       4.560
x2             1.0528      0.296      3.552      0.009       0.352       1.754
x3             0.4464      0.287      1.555      0.164      -0.233       1.125
==============================================================================
Omnibus:                        2.262   Durbin-Watson:                   2.981
Prob(Omnibus):                  0.323   Jarque-Bera (JB):                0.867
Skew:                          -0.059   Prob(JB):                        0.648
Kurtosis:                       1.562   Cond. No.                         6.98
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
# A) Test univariante: totchr = 0 (equivale a `test totchr=0`)
t_test_totchr = results.t_test('x3 = 1')
print("\nTest x3 = 1:")
print(t_test_totchr)


Test x3 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.4464      0.287     -1.928      0.095      -0.233       1.125


In [ ]:
# Test F equivalente (aunque sea una sola restricción)
f_test_result = results.f_test("x3 = 1")
print(f_test_result)

<F test: F=3.717906856224962, p=0.0951759293690687, df_denom=7, df_num=1>


In [ ]:
y = df['y']
x = df[['x2', 'x3']]
X = sm.add_constant(x)

# Generar el vector de pesos
w = np.exp(-1 + 0.2 * df['x2'])



# Ajustar el modelo WLS
model = sm.WLS(y, X, weights=1/w)
results = model.fit()

# Mostrar resumen del modelo
print(results.summary())

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.838
Model:                            WLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     18.08
Date:                Wed, 07 May 2025   Prob (F-statistic):            0.00172
Time:                        21:25:49   Log-Likelihood:                -25.951
No. Observations:                  10   AIC:                             57.90
Df Residuals:                       7   BIC:                             58.81
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6061      1.027      0.590      0.5

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [20]:
# Realizar el test de hipótesis: H0: coeficiente de x3 = 1
f_test_result = results.f_test("x3 = 1")
print(f_test_result)


<F test: F=3.656912567307812, p=0.09742655997334931, df_denom=7, df_num=1>


In [21]:
y = df['y']
x = df[['x2', 'x3']]
X = sm.add_constant(x)

# Ajustar el modelo OLS
model_ols = sm.OLS(y, X).fit()

# Obtener los residuos
uhat = model_ols.resid

# Calcular el logaritmo de los residuos al cuadrado
luhatsq = np.log(uhat ** 2)

# Ajustar el modelo OLS de luhatsq sobre x2
X_var = sm.add_constant(df['x2'])
model_var = sm.OLS(luhatsq, X_var).fit()

# Obtener los residuos del modelo de varianza
lvaru = model_var.resid

# Estimar las varianzas
varu = np.exp(lvaru)

# Ajustar el modelo WLS con pesos inversos a las varianzas estimadas
model_wls = sm.WLS(y, X, weights=1 / varu).fit()


print(model_wls.summary())

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.907
Model:                            WLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     34.31
Date:                Wed, 07 May 2025   Prob (F-statistic):           0.000241
Time:                        21:29:55   Log-Likelihood:                -24.175
No. Observations:                  10   AIC:                             54.35
Df Residuals:                       7   BIC:                             55.26
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4598      0.788      1.853      0.1

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [22]:
# Realizar el test F para H0: coeficiente de x3 = 1
f_test_result = model_wls.f_test("x3 = 1")
print(f_test_result)

<F test: F=14.0002570441167, p=0.0072466633098330294, df_denom=7, df_num=1>


In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pyreadstat as st
import numpy as np
import matplotlib.pyplot as plt


path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 1-20250507\data\cornwell.dta"

df, meta = st.read_dta(path)
df.head()

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,west,central,urban,pctmin80,wcon,wtuc,wtrd,wfir,wser,wmfg,wfed,wsta,wloc,mix,pctymle,d82,d83,d84,d85,d86,d87,lcrmrte,lprbarr,lprbconv,lprbpris,lavgsen,lpolpc,ldensity,ltaxpc,lwcon,lwtuc,lwtrd,lwfir,lwser,lwmfg,lwfed,lwsta,lwloc,lmix,lpctymle,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,0,1,0,20.2187,206.480301,333.620880,182.333008,272.449188,215.733490,229.119995,409.369995,236.240005,231.470001,0.099918,0.087697,0,0,0,0,0,0,-3.221757,-1.238923,-0.911149,-0.750306,1.724551,-6.327340,0.836017,3.246399,5.330205,5.810005,5.205835,5.607452,5.374044,5.434246,6.014619,5.464848,5.444450,-2.303407,-2.433870,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,0,1,0,20.2187,212.754196,369.296417,189.541412,300.878754,231.576706,240.330002,419.700012,253.880005,236.789993,0.103049,0.086377,1,0,0,0,0,0,-3.261134,-1.084381,-0.837006,-0.679258,1.720979,-6.338704,0.845977,3.213833,5.360137,5.911600,5.244607,5.706707,5.444911,5.482013,6.039540,5.536862,5.467174,-2.272549,-2.449038,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,0,1,0,20.2187,219.780212,1394.803467,196.639465,309.969635,240.156845,269.700012,438.850006,250.360001,248.580002,0.080679,0.085091,0,1,0,0,0,0,-3.496449,-1.107303,-0.643019,-0.734584,1.757858,-6.300291,0.850920,3.275311,5.392628,7.240509,5.281372,5.736475,5.481292,5.597310,6.084157,5.522900,5.515765,-2.517281,-2.464036,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,0,1,0,20.2187,223.423798,398.860413,200.562912,350.086334,252.447678,281.739990,459.170013,261.929993,264.380005,0.078504,0.083833,0,0,1,0,0,0,-3.360270,-1.014662,-0.503013,-0.653727,1.930071,-6.273361,0.852891,3.289981,5.409070,5.988612,5.301128,5.858180,5.531204,5.640985,6.129421,5.568077,5.577387,-2.544612,-2.478925,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,0,1,0,20.2187,243.756241,358.783020,206.882736,383.070740,261.086121,298.880005,490.429993,281.440002,288.579987,0.093249,0.082306,0,0,0,1,0,0,-3.308445,-1.122715,-0.546931,-0.699047,1.879465,-6.253162,0.860734,3.337204,5.496169,5.882718,5.332152,5.948220,5.564850,5.700042,6.195282,5.639919,5.664972,-2.372487,-2.497306,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   county    630 non-null    int64  
 1   year      630 non-null    int64  
 2   crmrte    630 non-null    float64
 3   prbarr    630 non-null    float64
 4   prbconv   630 non-null    float64
 5   prbpris   630 non-null    float64
 6   avgsen    630 non-null    float64
 7   polpc     630 non-null    float64
 8   density   630 non-null    float64
 9   taxpc     630 non-null    float64
 10  west      630 non-null    int64  
 11  central   630 non-null    int64  
 12  urban     630 non-null    int64  
 13  pctmin80  630 non-null    float64
 14  wcon      630 non-null    float64
 15  wtuc      630 non-null    float64
 16  wtrd      630 non-null    float64
 17  wfir      630 non-null    float64
 18  wser      630 non-null    float64
 19  wmfg      630 non-null    float64
 20  wfed      630 non-null    float6

In [ ]:
from linearmodels.panel.data import PanelData

df1 = df.set_index(['county', 'year']).sort_index()

# Crea el objeto PanelData
pdata = PanelData(df1)

# Información “estilo xtdescribe”
n_entities = len(pdata.entity_ids)
n_time     = len(pdata.time_ids)
n_obs      = pdata.dataframe.shape[0]

# Cálculo manual de “balanced”
counts_per_entity = pdata.dataframe.groupby(level='county').size()
min_p, max_p      = counts_per_entity.min(), counts_per_entity.max()
is_balanced       = (min_p == max_p)

print(f"Panel identifier:      county")
print(f"Time variable:         year")
print(f"Entidades (paneles):   {n_entities}")
print(f"Time points:           {n_time}")
print(f"Total observaciones:   {n_obs}")
print(f"Períodos por panel:    min={min_p}, max={max_p}")
print(f"Balanced panel?        {'Sí' if is_balanced else 'No'}")

Panel identifier:      county
Time variable:         year
Entidades (paneles):   630
Time points:           630
Total observaciones:   630
Períodos por panel:    min=7, max=7
Balanced panel?        Sí


In [11]:
pdata

PanelData
               crmrte    prbarr   prbconv   prbpris  avgsen     polpc  \
county year                                                             
1      81    0.039885  0.289696  0.402062  0.472222    5.61  0.001787   
       82    0.038345  0.338111  0.433005  0.506993    5.59  0.001767   
       83    0.030305  0.330449  0.525703  0.479705    5.80  0.001836   
       84    0.034726  0.362525  0.604706  0.520104    6.89  0.001886   
       85    0.036573  0.325395  0.578723  0.497059    6.55  0.001924   
...               ...       ...       ...       ...     ...       ...   
197    83    0.015575  0.226667  0.480392  0.428571    7.77  0.001073   
       84    0.013662  0.204188  1.410260  0.372727   10.11  0.001109   
       85    0.013086  0.180556  0.830769  0.333333    5.96  0.001054   
       86    0.012874  0.112676  2.250000  0.244444    7.68  0.001088   
       87    0.014193  0.207595  1.182930  0.360825   12.23  0.001186   

              density      taxpc  west  central  urban  pctmin80        wcon  \
county year                                                                    
1      81    2.307159  25.697630   0.0      1.0    0.0  20.21870  206.480301   
       82    2.330254  24.874252   0.0      1.0    0.0  20.21870  212.754196   
       83    2.341801  26.451443   0.0      1.0    0.0  20.21870  219.780212   
       84    2.346420  26.842348   0.0      1.0    0.0  20.21870  223.423798   
       85    2.364896  28.140337   0.0      1.0    0.0  20.21870  243.756241   
...               ...        ...   ...      ...    ...       ...         ...   
197    83    0.869048  18.905853   1.0      0.0    0.0   5.46081  207.100586   
       84    0.872024  22.704754   1.0      0.0    0.0   5.46081  230.227524   
       85    0.875000  24.123611   1.0      0.0    0.0   5.46081  253.036438   
       86    0.880952  24.981979   1.0      0.0    0.0   5.46081  280.554993   
       87    0.889881  25.952581   1.0      0.0    0.0   5.46081  314.166046   

                    wtuc        wtrd        wfir        wser        wmfg  \
county year                                                                
1      81     333.620880  182.333008  272.449188  215.733490  229.119995   
       82     369.296417  189.541412  300.878754  231.576706  240.330002   
       83    1394.803467  196.639465  309.969635  240.156845  269.700012   
       84     398.860413  200.562912  350.086334  252.447678  281.739990   
       85     358.783020  206.882736  383.070740  261.086121  298.880005   
...                  ...         ...         ...         ...         ...   
197    83     317.989105  154.320984  254.865616  196.463654  256.190002   
       84     304.878052  170.595535  262.237762  192.678223  268.589996   
       85     283.400818  171.673813  271.739136  207.257401  279.170013   
       86     324.374420  180.092712  312.294556  215.269806  306.089996   
       87     341.880341  182.801987  348.143250  212.820511  322.920013   

                   wfed        wsta        wloc       mix   pctymle  d82  d83  \
county year                                                                     
1      81    409.369995  236.240005  231.470001  0.099918  0.087697  0.0  0.0   
       82    419.700012  253.880005  236.789993  0.103049  0.086377  1.0  0.0   
       83    438.850006  250.360001  248.580002  0.080679  0.085091  0.0  1.0   
       84    459.170013  261.929993  264.380005  0.078504  0.083833  0.0  0.0   
       85    490.429993  281.440002  288.579987  0.093249  0.082306  0.0  0.0   
...                 ...         ...         ...       ...       ...  ...  ...   
197    83    352.649994  345.269989  235.899994  0.113861  0.079020  0.0  1.0   
       84    345.899994  354.209991  246.889999  0.116959  0.078233  0.0  0.0   
       85    357.839996  354.779999  275.010010  0.081081  0.077222  0.0  0.0   
       86    376.540009  355.640015  294.489990  0.075758  0.075701  0.0  0.0   
       87    391.720001  385.649994  3

In [12]:
# Falta abrir entre overall, between y within.

In [16]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

# Carga tus datos:
# df = pd.read_stata('ruta/datos.dta')
df = df.set_index(['county', 'year'])

y = df['lcrmrte']
X = df[['lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']]
X = sm.add_constant(X)

model = PooledOLS(y, X)
results = model.fit()
print(results.summary)

                          PooledOLS Estimation Summary                          
Dep. Variable:                lcrmrte   R-squared:                        0.5658
Estimator:                  PooledOLS   R-squared (Between):              0.6455
No. Observations:                 630   R-squared (Within):              -0.2685
Date:                Thu, May 08 2025   R-squared (Overall):              0.5658
Time:                        00:26:14   Log-likelihood                   -279.57
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      162.65
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(5,624)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             162.65
                            

In [ ]:
y = df['lcrmrte']
regressors = ['lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc',
              'd82', 'd83', 'd84', 'd85', 'd86', 'd87']
X = df[regressors]
X = sm.add_constant(X)

# 3) Ajusta Pooled OLS con errores robustos
model = PooledOLS(y, X)
results = model.fit()

# 4) Imprime el resumen
print(results.summary)

                          PooledOLS Estimation Summary                          
Dep. Variable:                lcrmrte   R-squared:                        0.5700
Estimator:                  PooledOLS   R-squared (Between):              0.6460
No. Observations:                 630   R-squared (Within):              -0.2255
Date:                Thu, May 08 2025   R-squared (Overall):              0.5700
Time:                        00:28:18   Log-likelihood                   -276.51
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.485
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(11,618)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             74.485
                            

In [27]:
y = df['lcrmrte']
regressors = ['lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc',
              'd82', 'd83', 'd84', 'd85', 'd86', 'd87']
X = sm.add_constant(df[regressors])

model = PooledOLS(y, X)
# cluster_entity=True → agrupa por el índice de entidad (county)
results = model.fit(cov_type='clustered', cluster_entity=True)

print(results.summary)

                          PooledOLS Estimation Summary                          
Dep. Variable:                lcrmrte   R-squared:                        0.5700
Estimator:                  PooledOLS   R-squared (Between):              0.6460
No. Observations:                 630   R-squared (Within):              -0.2255
Date:                Thu, May 08 2025   R-squared (Overall):              0.5700
Time:                        00:39:28   Log-likelihood                   -276.51
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      74.485
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(11,618)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             37.552
                            

In [ ]:
# Test de correlación serial. 
import statsmodels.api as sm
from scipy import stats

# 1. Carga de datos
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 1-20250507\data\cornwell.dta" 

# 1. Carga tus datos
df, _ = st.read_dta(path) # Ajusta ruta/nombre

# 2. Selección de variables y limpieza de NAs para el modelo original
vars_orig = [
    "lcrmrte",
    "lprbarr", "lprbconv", "lprbpris", "lavgsen", "lpolpc",
    "d82", "d83", "d84", "d85", "d86", "d87"
]
df_model = df.dropna(subset=vars_orig).copy()

# 3. Definir y (dependiente) y X (regresoras + dummies)
y = df_model["lcrmrte"]
X = df_model[[
    "lprbarr", "lprbconv", "lprbpris",
    "lavgsen", "lpolpc",
    "d82", "d83", "d84", "d85", "d86", "d87"
]]
X = sm.add_constant(X)

# 4. Ajuste del modelo POLS original (igual a `reg lcrmrte …` en Stata)
mod_orig = sm.OLS(y, X)
res_orig = mod_orig.fit()
print("=== Modelo POLS original ===")
print(res_orig.summary())

# 5. Almacenar residuos (igual a `predict uit, resid`)
df_model["uit"] = res_orig.resid

# 6. Construir el lag de los residuos por panel (igual a `l.uit`)
#    Si tienes columnas "county" y "year", agrupas por "county":
df_model["uit_lag"] = df_model.groupby("county")["uit"].shift(1)

# 7. Filtrar filas que tengan uit_lag y limpiar NAs en las mismas regresoras
aux_vars = [
    "uit",           # dependiente auxiliar
    "lprbarr", "lprbconv", "lprbpris", "lavgsen", "lpolpc",
    "d82", "d83", "d84", "d85", "d86", "d87",
    "uit_lag"
]
df_aux = df_model.dropna(subset=aux_vars).copy()

# 8. Definir y_aux y X_aux (igual a `reg uit … l.uit`)
y_aux = df_aux["uit"]
X_aux = df_aux[[
    "lprbarr", "lprbconv", "lprbpris",
    "lavgsen", "lpolpc",
    "d82", "d83", "d84", "d85", "d86", "d87",
    "uit_lag"
]]
X_aux = sm.add_constant(X_aux)

# 9. Ajuste de la regresión auxiliar
mod_aux = sm.OLS(y_aux, X_aux)
res_aux = mod_aux.fit()
print("\n=== Regresión auxiliar ===")
print(res_aux.summary())

# 10. Construir el estadístico LM = R² * N (igual a `local LM = e(r2)*e(N)`)
r2   = res_aux.rsquared
N    = res_aux.nobs   # número de obs usadas en la auxiliar
LM   = r2 * N
pval = stats.chi2.sf(LM, df=1)

# 11. Mostrar χ²(1) y p-valor (igual a los disp in red de Stata)
print(f"\nchi2(1) = {LM:.6f}")
print(f"El p-valor (LM) es {pval:.6f}. "
      f"{'Rechaza H0 al 5%' if pval < 0.05 else 'No rechaza H0 al 5%'}")

=== Modelo POLS original ===
                            OLS Regression Results                            
Dep. Variable:                lcrmrte   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     74.49
Date:                Fri, 09 May 2025   Prob (F-statistic):          1.38e-105
Time:                        21:43:12   Log-Likelihood:                -276.51
No. Observations:                 630   AIC:                             577.0
Df Residuals:                     618   BIC:                             630.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0823  

In [30]:
import pandas as pd
import statsmodels.api as sm
from scipy import stats

# 1) Carga de datos y limpieza
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 1-20250507\data\cornwell.dta" 

# 1. Carga tus datos
df, _ = st.read_dta(path)

df = df.dropna(subset=[
    "lcrmrte",
    "lprbarr", "lprbconv", "lprbpris", "lavgsen", "lpolpc",
    "d82", "d83", "d84", "d85", "d86", "d87",
    "county", "year"
])

# 2) Ajuste del modelo POLS original para obtener residuos (uit)
# Definir y y X para el modelo original
y = df["lcrmrte"]
X = df[[
    "lprbarr", "lprbconv", "lprbpris",
    "lavgsen", "lpolpc",
    "d82", "d83", "d84", "d85", "d86", "d87"
]]
X = sm.add_constant(X)

# Estimar modelo original
orig_mod = sm.OLS(y, X).fit()
df["uit"] = orig_mod.resid

# 3) Generar squit = uit^2
df["squit"] = df["uit"] ** 2

# 4) Crear los términos polinomiales y de interacción
df["lprbarr_sq"] = df["lprbarr"] ** 2
df["lprbpris_sq"] = df["lprbpris"] ** 2
df["cross"]      = df["lprbarr"] * df["lprbpris"]

# 5) Filtrar filas con NA en las variables de la prueba auxiliar
vars_aux = ["squit", "lprbarr", "lprbarr_sq", "lprbpris", "lprbpris_sq", "cross"]
df_aux = df.dropna(subset=vars_aux).copy()

# 6) Regresión auxiliar (White test)
y_aux = df_aux["squit"]
X_aux = df_aux[[
    "lprbarr", "lprbarr_sq",
    "lprbpris", "lprbpris_sq",
    "cross"
]]
X_aux = sm.add_constant(X_aux)

res_aux = sm.OLS(y_aux, X_aux).fit()

# 7) Cálculo del estadístico LM y p-valor con 5 grados de libertad
r2 = res_aux.rsquared
N  = res_aux.nobs
LM = r2 * N
p_value = stats.chi2.sf(LM, df=5)

# 8) Mostrar resultados
print("=== Modelo POLS original (para generar uit) ===")
print(orig_mod.summary())
print("\n=== Regresión auxiliar (White test) ===")
print(res_aux.summary())
print(f"\nchi2(5) = {LM:.6f}")
print(f"El p-valor (LM) es {p_value:.6f}. "
      f"{'Rechaza H0 al 5%' if p_value < 0.05 else 'No rechaza H0 al 5%'}")

=== Modelo POLS original (para generar uit) ===
                            OLS Regression Results                            
Dep. Variable:                lcrmrte   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     74.49
Date:                Fri, 09 May 2025   Prob (F-statistic):          1.38e-105
Time:                        21:57:10   Log-Likelihood:                -276.51
No. Observations:                 630   AIC:                             577.0
Df Residuals:                     618   BIC:                             630.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons

In [39]:
# Falta obtener el test Prais-Winsten